In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Github/3DQSAR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Github/3DQSAR


In [2]:
# !pip install --upgrade batchgenerators

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization,Flatten,\
Add,Input,Dense, Dropout, Activation, InputLayer, Concatenate
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import KFold
from scipy.ndimage import gaussian_filter

# from augmentation_code import data_augmentation
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os 
import pickle
import scipy
import random

In [ ]:
X_1,X_2,Y = pickle.load(open("/content/drive/My Drive/Github/data.p", "rb"))

In [ ]:
new_X = scipy.ndimage.rotate(X_1[0],90,reshape=False,order=3)
#random.choice([90,180,270])

In [ ]:
mol5

(array([44044, 44045, 44046, ..., 45042, 45043, 45044]),)

In [ ]:
Y[44044]

8.7

In [ ]:
np.linspace(Y.min(),Y.max(),10)

array([5.043351  , 5.44964533, 5.85593967, 6.262234  , 6.66852833,
       7.07482267, 7.481117  , 7.88741133, 8.29370567, 8.7       ])

In [ ]:
np.sort(Y[::1001])

array([5.043351, 5.154902, 5.19382 , 5.199971, 5.2     , 5.200659,
       5.250264, 5.260428, 5.30103 , 5.318759, 5.337242, 5.387216,
       5.39    , 5.497573, 5.58838 , 5.59998 , 5.657577, 5.699622,
       5.72    , 5.769551, 5.815309, 5.853872, 5.886057, 5.9     ,
       6.008774, 6.091515, 6.102373, 6.195861, 6.200659, 6.32    ,
       6.346787, 6.37    , 6.400117, 6.413413, 6.42    , 6.537602,
       6.542118, 6.548214, 6.55    , 6.55752 , 6.63    , 6.69897 ,
       6.886057, 7.      , 7.      , 7.      , 7.1     , 7.28    ,
       7.809668, 7.85    , 7.9     , 8.      , 8.102373, 8.7     ])

In [ ]:
5.19382,5.497573,5.699622,5.9,6.346787,6.102373,6.63,7.,7.9,8.

In [ ]:
X_1.shape

(54054, 14, 14, 14)

In [ ]:
z=np.squeeze(test_index).shape

In [ ]:
np.delete(X_1,z,0).shape

(54053, 14, 14, 14)

In [ ]:
# X_1, X_2, Y = import_data()

In [ ]:
####Augment and extend data
# X_1_train, X_2_train, Y_train = data_augmentation(X_1_train,X_2_train,Y_train,augment_repetitions=2)

In [ ]:
###Pickle data

In [ ]:
###Normalise image data 
###Normalise by amax value?

In [4]:
def split_shuffle(X_1,X_2,Y,augment,aug_iter):
    ###Reshape
    X_1 = np.reshape(X_1,(X_1.shape[0],X_1.shape[1],X_1.shape[2],X_1.shape[3],1))
    X_2 = np.reshape(X_2,(X_2.shape[0],X_2.shape[1],X_2.shape[2],X_2.shape[3],1))
    
    mol1 = np.where(Y==(5.154902))
    mol2 = np.where(Y==(5.815309))
    mol3 = np.where(Y==(6.69897))
    mol4 = np.where(Y==(7.85))
    mol5 = np.where(Y==(8.102373)) 

    test_index = np.hstack([mol1,mol2,mol3,mol4,mol5])

    test_index = np.squeeze(test_index)

    X_1_train = np.delete(X_1,test_index,0)
    X_2_train = np.delete(X_2,test_index,0)
    X_1_test = X_1[test_index]
    X_2_test = X_2[test_index]

    Y_train = np.delete(Y,test_index)
    Y_test = Y[test_index]

           
    mol1 = np.where(Y_train==(5.19382))
    mol2 = np.where(Y_train==(5.497573))
    mol3 = np.where(Y_train==(5.699622))
    mol4 = np.where(Y_train==(5.9))
    mol5 = np.where(Y_train==(6.346787))

    mol6 = np.where(Y_train==(6.102373))
    mol7 = np.where(Y_train==(6.63))
    mol8 = np.where(Y_train==(7.1))
    mol9 = np.where(Y_train==(7.9))
    mol10 = np.where(Y_train==(8.)) 
    
    val_index = np.hstack([mol1,mol2,mol3,mol4,mol5,
                           mol6,mol7,mol8,mol9,mol10])
    
    val_index = np.squeeze(val_index)
  
    X_1_val = X_1_train[val_index]
    X_2_val = X_2_train[val_index]
    X_1_train = np.delete(X_1_train, val_index, 0)
    X_2_train = np.delete(X_2_train, val_index, 0)

    Y_val = Y_train[val_index]
    Y_train = np.delete(Y_train, val_index)

    
    permutation_train = np.random.RandomState(seed=42)\
                       .permutation(X_1_train.shape[0])
    permutation_test = np.random.RandomState(seed=42)\
                       .permutation(X_1_test.shape[0])
    permutation_val = np.random.RandomState(seed=42)\
                       .permutation(X_1_val.shape[0])

    split_train = int(np.around(X_1_train.shape[0]/4,0))
    split_val = int(np.around(X_1_val.shape[0]/4,0))
    split_test = int(np.around(X_1_test.shape[0]/4,0))

    X_1_val =   X_1_val[permutation_val]#[:split_val] 
    X_2_val =   X_2_val[permutation_val]#[:split_val] 
    X_1_train = X_1_train[permutation_train]#[:split_train]
    X_2_train = X_2_train[permutation_train]#[:split_train]
    Y_val =     Y_val[permutation_val]#[:split_val]
    Y_train =   Y_train[permutation_train]#[:split_train] 
    X_1_test =  X_1_test[permutation_test]#[:split_test]
    X_2_test =  X_2_test[permutation_test]#[:split_test]
    Y_test =    Y_test[permutation_test]#[:split_test]
   


    if augment and aug_iter!=0:
      indexes = np.where(Y_train>7.25)
      for _ in np.arange(aug_iter):
        
        # new_X = scipy.ndimage.gaussian_filter(X_1_train[indexes],sigma=(_+1))
        # X_1_train = np.concatenate([X_1_train,new_X])

        # new_X = scipy.ndimage.gaussian_filter(X_2_train[indexes],sigma=(_+1))
        # X_2_train = np.concatenate([X_2_train,new_X])

        # Y_train = np.concatenate([Y_train,Y_train[indexes]])

        new_X = scipy.ndimage.rotate(X_1_train[indexes],random.choice([90,180,270]),reshape=False,order=3)
        X_1_train = np.concatenate([X_1_train,new_X])

        new_X = scipy.ndimage.rotate(X_2_train[indexes],random.choice([90,180,270]),reshape=False,order=3)
        X_2_train = np.concatenate([X_2_train,new_X])

        Y_train = np.concatenate([Y_train,Y_train[indexes]])
    else:
      print("No augmentation!")  

    return X_1_train, X_1_test, X_1_val, X_2_train, X_2_test, X_2_val, Y_train, Y_test, Y_val

In [42]:
def build_CNN(X_1_train,layer):
    model = Sequential(name='CNN')
    
    # elu=Exponential Linear Unit, similar to leaky Relu
    #perhaps normalise in layer 1 
    
    # Convolution Layers
    tf.random.set_seed(42)
    np.random.seed(42)
    inputs = Input(shape=(X_1_train.shape[1], X_1_train.shape[2],X_1_train.shape[3],X_1_train.shape[4]))
    model=inputs
    model=BatchNormalization()(model)
    model = Conv3D(32, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    # model = MaxPooling3D(pool_size=(2,2,2))(model)
    # model=BatchNormalization()(model)
    model = Conv3D(64, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    model = MaxPooling3D(pool_size=(2,2,2))(model)
    model = Conv3D(128, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    # model = MaxPooling3D(pool_size=(1,1,1))(model)
    # model=BatchNormalization()(model)
    model = Flatten()(model)
    ####increase
    model = Dense(100,activation='relu')(model)
    output = model

    model = Model(inputs=inputs, outputs=output)
    
    return model

In [43]:
def two_headed_network(X_1_train,layer):
    tf.random.set_seed(42)
    np.random.seed(42)
    X_1_CNN = build_CNN(X_1_train,layer)
    X_2_CNN = build_CNN(X_1_train,layer)
    network = Concatenate()([X_1_CNN.output,X_2_CNN.output])
    network = Add()([X_1_CNN.output,X_2_CNN.output])
    # if layer=="add":
    #   network = Add()([X_1_CNN.output,X_2_CNN.output])
    #   print(layer)
    # elif layer=="concat":
    #   network = Concatenate()([X_1_CNN.output,X_2_CNN.output])
    #   print(layer)
    
    network = Flatten()(network)
    network = Dense(100,activation='relu')(network)
    network = Dense(50,activation='relu')(network)
    network = Dense(10,activation='relu')(network)
    network = Dense(1,activation = 'linear')(network)
    model = Model([X_1_CNN.input, X_2_CNN.input], network)
    optimizer = Adam(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer)
    
    return model  
# CNN = two_headed_network(X_1_train,None)

In [ ]:
CNN.summary()

In [ ]:
tf.keras.utils.plot_model(CNN)

In [7]:
# #####Cross validation
def train_CV(CNN,X_1_train, X_1_val, X_2_train,\
  X_2_val, Y_train, Y_val):
  perm = np.random.RandomState(seed=42)\
                  .permutation(X_1_train.shape[0])
  num_folds=4
  kfold = KFold(n_splits=num_folds, shuffle=False, random_state=42)
  monitor = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=7,
                          verbose=1, 
                          mode='auto',
                          restore_best_weights=True)

  for perms_train,perms_test in kfold.split(X=perm,y=perm):
      CNN.fit([X_1_train[perms_train], X_2_train[perms_train]],
                        Y_train[perms_train],
                        batch_size=32,
                        epochs=1000,
                        verbose=0, 
                        validation_data=([X_1_train[perms_test], X_2_train[perms_test]],
                                          Y_train[perms_test]),
                                          callbacks=[monitor])
  model_loss = CNN.evaluate([X_1_val,X_2_val], Y_val)
  return model_loss

In [8]:
##Train 
##monitor early stopping using val_acc
#Set min_delta to 1e-3
def train(CNN,X_1_train, X_1_val, X_2_train,\
  X_2_val, Y_train, Y_val, batch):

  monitor = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=15,
                          verbose=1, 
                          mode='auto',
                          restore_best_weights=True)
  tf.random.set_seed(42)
  np.random.seed(42)
  CNN.fit([X_1_train, X_2_train],
          Y_train,
          batch_size=batch,
          epochs=1000, 
          validation_data=([X_1_val, X_2_val],
                            Y_val),
                            callbacks=[monitor],
          shuffle=False,
          verbose=1)
  
  model_loss = CNN.evaluate([X_1_val,X_2_val], Y_val, verbose=0)
  return model_loss
#######################################
#Reset shuffle to True
#######################################

In [41]:
def experiment(itr,layer):
  X_1,X_2,Y = pickle.load(open("/content/drive/My Drive/Github/data.p", "rb"))

  X_1_train, X_1_test, X_1_val, X_2_train, X_2_test,\
  X_2_val, Y_train, Y_test, Y_val = split_shuffle(X_1,X_2,Y,augment=False,aug_iter=itr)
  del X_1,X_2,Y
  
  tf.random.set_seed(42)
  np.random.seed(42)

  CNN = two_headed_network(X_1_train,layer)
  print(X_1_train.shape)
  # if CV:
  #   loss = train_CV(CNN,X_1_train, X_1_val, X_2_train,\
  #                   X_2_val, Y_train, Y_val)
  batch=32
  loss = train(CNN,X_1_train, X_1_val, X_2_train,\
                 X_2_val, Y_train, Y_val, batch)
  print(loss)
  plt.hist(Y_train)
  return loss,CNN
  
  #1 iter = best

In [ ]:
##################Run experiment


losses=[]
layer=None
# for layer in ["","no"]:
for batch in [16,32,64,128]:
  itr_loss=0
  for _ in np.arange(3):
    loss,CNN = experiment(0,layer)
    itr_loss+=loss
    tf.keras.backend.clear_session()
  print(itr_loss/3)
  losses.append(itr_loss/5)
# with (open("results_conv_fc_concat.txt","w")) as file:
#   file.write(f"add{str(losses[0])}")
#   file.write("\n")
#   file.write(f"concat{str(losses[1])}")  

In [ ]:
#################### Run single test


layer=32
loss,CNN = experiment(0,layer)
itr_loss+=loss
tf.keras.backend.clear_session()


No augmentation!
(39039, 14, 14, 14, 1)
Epoch 1/1000
1220/1220 [==============================] - 12s 9ms/step - loss: 0.8138 - val_loss: 1.3523
Epoch 2/1000
1220/1220 [==============================] - 11s 9ms/step - loss: 0.2214 - val_loss: 1.7980
Epoch 3/1000
1220/1220 [==============================] - 11s 9ms/step - loss: 0.1293 - val_loss: 1.7081
Epoch 4/1000
1220/1220 [==============================] - 11s 9ms/step - loss: 0.0717 - val_loss: 1.8399
Epoch 5/1000
1220/1220 [==============================] - 11s 9ms/step - loss: 0.0545 - val_loss: 1.8583
Epoch 6/1000
 765/1220 [=================>............] - ETA: 3s - loss: 0.0408

In [23]:
CNN.predict([X_1_train,X_2_train])

array([[6.0280857],
       [6.624783 ],
       [6.4909406],
       ...,
       [5.956703 ],
       [6.545893 ],
       [6.244688 ]], dtype=float32)

In [36]:
preds = CNN.predict([X_1_val,X_2_val])

In [37]:
preds[:10]

array([[7.0426764],
       [5.2225046],
       [5.326735 ],
       [5.21741  ],
       [6.7216215],
       [5.3269157],
       [7.0873995],
       [6.986512 ],
       [5.55239  ],
       [7.1098957]], dtype=float32)

In [38]:
Y_val[:10]

array([7.1     , 8.      , 5.19382 , 8.      , 5.699622, 5.9     ,
       7.1     , 5.497573, 5.497573, 7.1     ])

In [ ]:
X_1_train.shape

(39039, 14, 14, 14, 1)

In [ ]:
X_1_val[permutation_val]

In [14]:
    X_1,X_2,Y = pickle.load(open("/content/drive/My Drive/Github/data.p", "rb"))
  
    
    X_1 = np.reshape(X_1,(X_1.shape[0],X_1.shape[1],X_1.shape[2],X_1.shape[3],1))
    X_2 = np.reshape(X_2,(X_2.shape[0],X_2.shape[1],X_2.shape[2],X_2.shape[3],1))
    
    mol1 = np.where(Y==(5.154902))
    mol2 = np.where(Y==(5.815309))
    mol3 = np.where(Y==(6.69897))
    mol4 = np.where(Y==(7.85))
    mol5 = np.where(Y==(8.102373)) 

    test_index = np.hstack([mol1,mol2,mol3,mol4,mol5])

    test_index = np.squeeze(test_index)

    X_1_train = np.delete(X_1,test_index,0)
    X_2_train = np.delete(X_2,test_index,0)
    X_1_test = X_1[test_index]
    X_2_test = X_2[test_index]

    Y_train = np.delete(Y,test_index)
    Y_test = Y[test_index]

           
    mol1 = np.where(Y_train==(5.19382))
    mol2 = np.where(Y_train==(5.497573))
    mol3 = np.where(Y_train==(5.699622))
    mol4 = np.where(Y_train==(5.9))
    mol5 = np.where(Y_train==(6.346787))

    mol6 = np.where(Y_train==(6.102373))
    mol7 = np.where(Y_train==(6.63))
    mol8 = np.where(Y_train==(7.1))
    mol9 = np.where(Y_train==(7.9))
    mol10 = np.where(Y_train==(8.)) 
    
    val_index = np.hstack([mol1,mol2,mol3,mol4,mol5,
                           mol6,mol7,mol8,mol9,mol10])
    
    val_index = np.squeeze(val_index)

    X_1_val = X_1_train[val_index]
    X_2_val = X_2_train[val_index]
    X_1_train = np.delete(X_1_train, val_index, 0)
    X_2_train = np.delete(X_2_train, val_index, 0)

    Y_val = Y_train[val_index]
    Y_train = np.delete(Y_train, val_index)

    
    permutation_train = np.random.RandomState(seed=42)\
                       .permutation(X_1_train.shape[0])
    permutation_test = np.random.RandomState(seed=42)\
                       .permutation(X_1_test.shape[0])
    permutation_val = np.random.RandomState(seed=42)\
                       .permutation(X_1_val.shape[0])

    split_train = int(np.around(X_1_train.shape[0]/4,0))
    split_val = int(np.around(X_1_val.shape[0]/4,0))
    split_test = int(np.around(X_1_test.shape[0]/4,0))

    X_1_val =   X_1_val[permutation_val]#[:split_val] 
    X_2_val =   X_2_val[permutation_val]#[:split_val] 
    X_1_train = X_1_train[permutation_train]#[:split_train]
    X_2_train = X_2_train[permutation_train]#[:split_train]
    Y_val =     Y_val[permutation_val]#[:split_val]
    Y_train =   Y_train[permutation_train]#[:split_train] 
    X_1_test =  X_1_test[permutation_test]#[:split_test]
    X_2_test =  X_2_test[permutation_test]#[:split_test]
    Y_test =    Y_test[permutation_test]#[:split_test]
   

In [ ]:
[1.4205801904201507, 0.6195118814706803]


[1.4205801904201507, 0.6195118814706803]

In [ ]:
param_dict= {"batch_norm":(True,False),
             "dropout":(0.2,0.15,0.1),
             "l1":(0.001,0.01,0.02),
             "l2":(0.001,0.01,0.02),
             "l1_l2":(0.001,0.01,0.02),
             "lr":(1e-3,2e-3,1.5e-3),
             "batch_size":(16,32,64,128),
             "acivation":("relu","elu","")
            }

In [ ]:
 ##Test model w/ best weights on validation data
model_loss = CNN.evaluate([X_1_val,X_2_val], Y_val)
print(f"Model loss:{model_loss}")


305/305 [==============================] - 1s 3ms/step - loss: 0.1351
Model loss:0.13511130213737488


In [ ]:
X_1,X_2,Y = pickle.load(open("/content/drive/My Drive/Github/data.p", "rb"))

X_1_train, X_1_test, X_1_val, X_2_train, X_2_test,\
X_2_val, Y_train, Y_test, Y_val = split_shuffle(X_1,X_2,Y,augment=False,aug_iter=0)
# del X_1,X_2,Y

CNN = two_headed_network(X_1_train)
print(X_1_train.shape)
loss = train(CNN,X_1_train, X_1_val, X_2_train,\
X_2_val, Y_train, Y_val)

No augmentation!
(38919, 14, 14, 14, 1)
305/305 [==============================] - 1s 3ms/step - loss: 1.0267
